In [1]:
import pandas as pd
from faker import Faker
import random
from functools import reduce

In [2]:
def format_harga(harga):
    return f"IDR {harga:,.0f}".replace(",", ".")

In [3]:
def generate_data(num_days=4, products_per_day=3):
    faker = Faker()
    dates = pd.date_range("2024-08-06", periods=num_days).strftime('%Y-%m-%d')

    food_names = [
        f"{faker.word()} {random.choice(['Pizza', 'Burger', 'Pasta', 'Salad', 'Sushi', 'Soup', 'Steak', 'Sandwich'])}"
        for _ in range(products_per_day * num_days)
    ]

    data_penjualan = pd.DataFrame({
        "id": ["GN" + str(random.randint(100, 999)) for _ in range(products_per_day * num_days)],
        "nama": food_names,
        "harga": [random.randint(20000, 200000) for _ in range(products_per_day * num_days)],
        "quantity": [random.randint(1, 100) for _ in range(products_per_day * num_days)],
        "tanggal": [date for date in dates for _ in range(products_per_day)]
    })

    data_penjualan['harga_formatted'] = list(map(format_harga, data_penjualan['harga']))
    return data_penjualan

data_penjualan = generate_data()
print(data_penjualan[['id', 'nama', 'harga_formatted', 'quantity', 'tanggal']])

       id              nama harga_formatted  quantity     tanggal
0   GN753  without Sandwich      IDR 23.651        81  2024-08-06
1   GN727         only Soup      IDR 84.573         3  2024-08-06
2   GN783       power Sushi      IDR 74.841        56  2024-08-06
3   GN218       piece Salad      IDR 59.827        27  2024-08-07
4   GN473       night Sushi      IDR 78.720        43  2024-08-07
5   GN832      serve Burger      IDR 44.384        96  2024-08-07
6   GN972       watch Sushi     IDR 186.981        55  2024-08-08
7   GN143      cup Sandwich     IDR 144.495        56  2024-08-08
8   GN124      garden Salad      IDR 31.908        19  2024-08-08
9   GN827   evidence Burger     IDR 145.057        54  2024-08-09
10  GN625         part Soup      IDR 54.895        15  2024-08-09
11  GN734      himself Soup      IDR 70.915        87  2024-08-09


In [4]:
def hitung_pendapatan(df):
    df['pendapatan'] = df['harga'] * df['quantity']
    df['pendapatan_formatted'] = list(map(format_harga, df['pendapatan']))
    return df[['id', 'nama', 'harga_formatted', 'quantity', 'tanggal', 'pendapatan', 'pendapatan_formatted']]

data_with_pendapatan = hitung_pendapatan(data_penjualan)
print(data_with_pendapatan.to_string())

       id              nama harga_formatted  quantity     tanggal  pendapatan pendapatan_formatted
0   GN753  without Sandwich      IDR 23.651        81  2024-08-06     1915731        IDR 1.915.731
1   GN727         only Soup      IDR 84.573         3  2024-08-06      253719          IDR 253.719
2   GN783       power Sushi      IDR 74.841        56  2024-08-06     4191096        IDR 4.191.096
3   GN218       piece Salad      IDR 59.827        27  2024-08-07     1615329        IDR 1.615.329
4   GN473       night Sushi      IDR 78.720        43  2024-08-07     3384960        IDR 3.384.960
5   GN832      serve Burger      IDR 44.384        96  2024-08-07     4260864        IDR 4.260.864
6   GN972       watch Sushi     IDR 186.981        55  2024-08-08    10283955       IDR 10.283.955
7   GN143      cup Sandwich     IDR 144.495        56  2024-08-08     8091720        IDR 8.091.720
8   GN124      garden Salad      IDR 31.908        19  2024-08-08      606252          IDR 606.252
9   GN827 

In [5]:
def average_penjualan(df, tanggal, min_revenue=500000):
    try:
        # Filter rows based on date
        sales_on_date = df[df['tanggal'] == tanggal]
        if sales_on_date.empty:
            raise ValueError("Tanggal tidak ditemukan.")

        # Filter to include only rows where 'pendapatan' >= min_revenue
        high_revenue_sales = sales_on_date[sales_on_date['pendapatan'] >= min_revenue]
        
        # If no rows meet the condition, return a message
        if high_revenue_sales.empty:
            return f"Tidak ada penjualan dengan pendapatan di atas {format_harga(min_revenue)} pada tanggal tersebut."

        # Reduce to calculate total revenue by summing 'pendapatan' for filtered rows
        total_sales = reduce(int.__add__, (high_revenue_sales['pendapatan']))
        avg_sales = total_sales / len(high_revenue_sales)
        return format_harga(int(avg_sales))

    except ValueError as e:
        return str(e)
    except Exception as e:
        return "Format tanggal tidak sesuai."
    
tanggal_input = input("skibidi")
average = average_penjualan(data_penjualan, tanggal_input)
print(f"Rata-rata penjualan pada tanggal {tanggal_input}: {average}")

Rata-rata penjualan pada tanggal 2024-08-07: IDR 3.087.051


In [6]:
def total_penjualan_recursive(grouped_data, index=0, results=None):
    if results is None:
        results = []
    
    # Base case for recursion
    if index >= len(grouped_data):
        return results
    
    row = grouped_data.iloc[index]
    # Append result for the current date
    results.append({"tanggal": row['tanggal'], "total_penjualan": format_harga(int(row['pendapatan']))})
    
    # Recursive call with incremented index
    return total_penjualan_recursive(grouped_data, index + 1, results)

grouped = data_with_pendapatan.groupby('tanggal')['pendapatan'].sum().reset_index()
total_sales_by_date = total_penjualan_recursive(grouped)
for penjualan in total_sales_by_date:
    print(f"Tanggal: {penjualan['tanggal']}, Total Penjualan: {penjualan['total_penjualan']}")

Tanggal: 2024-08-06, Total Penjualan: IDR 6.360.546
Tanggal: 2024-08-07, Total Penjualan: IDR 9.261.153
Tanggal: 2024-08-08, Total Penjualan: IDR 18.981.927
Tanggal: 2024-08-09, Total Penjualan: IDR 14.826.108
